In [1]:
print("Setting CUDA devices...")
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="5"


from fastai.conv_learner import *
from fastai.dataset import *

import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split


Setting CUDA devices...


/home/ben/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
DATASET_ROOT = '/media/data-nvme/dev/datasets/airbus/'
PATH = DATASET_ROOT
TRAIN = DATASET_ROOT + 'train_v2/'
TEST = DATASET_ROOT + 'test_v2/'
SEGMENTATION = DATASET_ROOT + 'train_ship_segmentations_v2.csv'
exclude_list = ['6384c3e78.jpg'] #corrupted image

In [3]:
!rm -r {PATH}tmp # do that to avoid

In [4]:
nw = 12   #number of workers for data loader
arch = resnet34 #specify target architecture

In [5]:
train_names = [f for f in os.listdir(TRAIN)]
test_names = [f for f in os.listdir(TEST)]
for el in exclude_list:
    if(el in train_names): train_names.remove(el)
    if(el in test_names): test_names.remove(el)
#5% of data in the validation set is sufficient for model evaluation
tr_n, val_n = train_test_split(train_names, test_size=0.05, random_state=42)

In [6]:
class pdFilesDataset(FilesDataset):
    def __init__(self, fnames, path, transform):
        self.segmentation_df = pd.read_csv(SEGMENTATION).set_index('ImageId')
        super().__init__(fnames, transform, path)
    
    def get_x(self, i):
        img = open_image(os.path.join(self.path, self.fnames[i]))
        if self.sz == 768: return img 
        else: return cv2.resize(img, (self.sz, self.sz))
    
    def get_y(self, i):
        if(self.path == TEST): return 0
        masks = self.segmentation_df.loc[self.fnames[i]]['EncodedPixels']
        if(type(masks) == float): return 0 #NAN - no ship 
        else: return 1
    
    def get_c(self): return 2 #number of classes

In [7]:
def get_data(sz,bs):
    #data augmentation
    aug_tfms = [RandomRotate(20, tfm_y=TfmType.NO),
                RandomDihedral(tfm_y=TfmType.NO),
                RandomLighting(0.05, 0.05, tfm_y=TfmType.NO)]
    tfms = tfms_from_model(arch, sz, crop_type=CropType.NO, tfm_y=TfmType.NO, 
                aug_tfms=aug_tfms)
    ds = ImageData.get_ds(pdFilesDataset, (tr_n[:-(len(tr_n)%bs)],TRAIN), 
                (val_n,TRAIN), tfms, test=(test_names,TEST))
    md = ImageData(PATH, ds, bs, num_workers=nw, classes=None)
#     md.is_multi = False
    return md

In [8]:
sz = 256 #image size
bs = 64  #batch size

md = get_data(sz,bs)

In [9]:
learn = ConvLearner.pretrained(arch, md, ps=0.5) #dropout 50%
learn.opt_fn = optim.Adam

In [10]:
#learn.lr_find()
#learn.sched.plot()

In [11]:
#learn.fit(2e-3, 1)

In [12]:
learn.unfreeze()
lr=np.array([1e-4,5e-4,2e-3])

In [13]:
def save_when_acc(self, metrics):
    loss, acc = metrics[0], metrics[1]
    filename = "{}-l{:.3f}-a{:.3f}".format(self.name, loss[0], acc)
    if self.best_acc == None or acc > self.best_acc:
        self.best_acc = acc
        self.best_loss = loss
        self.model.save(f'{filename}')
    elif acc == self.best_acc and  loss < self.best_loss:
        self.best_loss = loss
        self.model.save(f'{filename}')
SaveBestModel.save_when_acc = save_when_acc

In [14]:
#learn.fit(lr, 1, cycle_len=2, use_clr=(20,8), best_save_name=arch.__name__ + "_kaggle-airbus")

In [15]:
#learn.sched.plot_lr()

In [16]:
#learn.save('Resnet34_lable_256_1')

## Do many times

In [ ]:
# A tester :
'''
[1, 2, (20,8), 384, 32],
'''
training_loop = [
    [1, 2, (20,8), 256, 64],
    [1, 2, (20,8), 256, 64],
    [2, 3, (15,4), 256, 64],
    [2, 3, (25,4), 256, 64],
    [1, 2, (25,10), 256, 64],
    [1, 2, (20,8), 512, 32]
]
i = 0
for epochs, cycle_len, use_clr, sz, bs in training_loop:
    i+=1
    md = get_data(sz,bs)
    learn = ConvLearner.pretrained(arch, md, ps=0.5) #dropout 50%
    learn.opt_fn = optim.Adam
    learn.fit(2e-3, 1, best_save_name=arch.__name__ + 's' + str(sz) + "_kaggle-airbus")
    learn.unfreeze()
    lr=np.array([1e-4,5e-4,2e-3])
    learn.fit(lr, epochs, cycle_len=cycle_len, use_clr=use_clr,
        best_save_name=arch.__name__ + 's' + str(sz) + "_kaggle-airbus")
    print("----------", i, "------------------------------")

epoch      trn_loss   val_loss   accuracy                      
    0      0.174303   0.145599   0.943498  



epoch      trn_loss   val_loss   accuracy                       
    0      0.096974   0.085757   0.967802  
  5%|▌         | 156/2858 [02:47<48:12,  1.07s/it, loss=0.0989] 

## Load the best

In [ ]:
log_preds,y = learn.predict_with_targs(is_test=True)
probs = np.exp(log_preds)[:,1]
pred = (probs > 0.5).astype(int)

In [ ]:
df = pd.DataFrame({'id':test_names, 'p_ship':probs})
df.to_csv('ship_detection.csv', header=True, index=False)

In [ ]:
sz = 384 #image size
bs = 32  #batch size

md = get_data(sz,bs)
learn = ConvLearner.pretrained(arch, md, ps=0.5) #dropout 50%
learn.opt_fn = optim.Adam
learn.unfreeze()
lr=np.array([1e-4,5e-4,2e-3])